In [77]:
import os
os.startfile("outlook")
import win32com.client as client
import pandas as pd
from pandas import DataFrame
import numpy as np
outlook = client.Dispatch('Outlook.Application')
namespace = outlook.GetNameSpace("MAPI")
account=namespace.Folders['vkk1693@gmail.com']
inbox=account.Folders['Inbox']
inbox.Items.count

3967

In [78]:
p=[]

for folder in inbox.Folders:
    p.append(folder.name)

fn=DataFrame(p,columns=['Main_Folder'])
fn['Sub_Folder']=''

In [79]:
z=[]
x=[]

for j in fn.index:
    for folder in inbox.Folders[fn['Main_Folder'][j]].Folders:
        x.append(fn['Main_Folder'][j])
        z.append(folder.Name)
        
new=pd.DataFrame(columns=['Main_Folder','Sub_Folder'])
new=pd.DataFrame(data={'Main_Folder':x,'Sub_Folder':z})

In [80]:
a=pd.concat([fn,new])
a=a.reset_index()
a=a.drop(columns=['index'])

In [82]:
e=[]
c=[]
d=[]
f=[]
g=[]

for k in a.index:
    if a['Sub_Folder'][k]=='':
        for email in inbox.Folders[a['Main_Folder'][k]].Items:
                            
            e.append(str(email.EntryID))
            c.append(str(email.Subject))
            d.append(email.ReceivedTime.strftime('%Y/%m/%d'))
            f.append(a['Main_Folder'][k])
            g.append('NA')


    else:
        for email in inbox.Folders[a['Main_Folder'][k]].Folders[a['Sub_Folder'][k]].Items:
            e.append(str(email.EntryID))
            c.append(str(email.Subject))
            d.append(email.ReceivedTime.strftime('%Y/%m/%d'))
            f.append(a['Main_Folder'][k])
            g.append(a['Sub_Folder'][k])


new=pd.DataFrame(columns=['Id','Subject','Received_Date','Main_Folder','Sub_Folder'])
new=pd.DataFrame(data={'Id':e,'Subject':c,'Received_Date':d,'Main_Folder':f,'Sub_Folder':g})

new['Today']=pd.datetime.today().strftime('%Y/%m/%d')
new['Today']=pd.to_datetime(new['Today'])
new['Received_Date']=pd.to_datetime(new['Received_Date'])
new['Days']=new['Today']-new['Received_Date']
new['Days']=new['Days'].astype('timedelta64[D]')
new['Days']=new['Days'].astype('int')

<ipython-input-82-028e60fe7602>:30: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  new['Today']=pd.datetime.today().strftime('%Y/%m/%d')


In [83]:
delete=new[new['Days']>30]
delete=delete.reset_index()
delete=delete.drop(columns=['index'])

In [84]:
delete_ws=delete[delete['Sub_Folder']!='NA']
delete_wos=delete[delete['Sub_Folder']=='NA']

In [85]:
delete_wos=delete_wos.sort_values(by='Main_Folder', ascending=True)
delete_ws=delete_ws.sort_values(by=['Main_Folder', 'Sub_Folder'])

In [86]:
delete_ws=delete_ws.reset_index()
delete_ws=delete_ws.drop(columns=['index'])

delete_wos=delete_wos.reset_index()
delete_wos=delete_wos.drop(columns=['index'])

In [87]:
x=delete_ws['Id'].tolist()
x=tuple(x)

y=delete_wos['Id'].tolist()
y=tuple(y)

In [88]:
try:
    account.Folders['Inbox'].Folders['tobedeleted'].Items.count
except:
    inbox.Folders.add('tobedeleted')

In [89]:
mf=delete_wos.groupby(['Main_Folder']).size().reset_index()
sf=delete_ws.groupby(['Main_Folder','Sub_Folder']).size().reset_index()

In [90]:
mf=mf[mf['Main_Folder']!='tobedeleted']

In [91]:
for k in sf.index:
    try:
        for message in [message for message in inbox.Folders[sf['Main_Folder'][k]].Folders[sf['Sub_Folder'][k]].Items if message.EntryID.startswith(x)]:
            message.Move(account.Folders['Inbox'].Folders['tobedeleted'])
    except:
        break

In [28]:
for k in mf.index:
    try:
        for message in [message for message in inbox.Folders[mf['Main_Folder'][k]].Items if message.EntryID.startswith(y)]:
            message.Move(account.Folders['Inbox'].Folders['tobedeleted'])
    except:
        break